[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Feng96/rasterarea/blob/master/docs/examples/test1.ipynb)

In [ ]:
%pip install rasterarea

In [1]:
import rasterarea

In [2]:
rasterarea.generate_password(8)

'"vRjAU_='

In [3]:
rasterarea.gnerate_lucky_number(1)

2